# Circuit Mapping Algorithms
---
### Basic Information
**Description:** This script defines the data structure of quantum hardware and a circuit mapping scoring function.\
**Note:** This circuit mapping scoring process is based on the [circuit fidelity estimation function](https://github.com/Qiskit-Partners/mapomatic/blob/main/mapomatic/layouts.py) of `MAPOMATIC`.

### Define a normal distribution of gate error rates in quantum hardware

In [312]:
import numpy as np

# Note: the coupler length is assumed to be 1 for all chip topology
def retrieve_node_coordinates(node_index, node_dict, graph_type):
    # construct a mapping from node index to coordinates
    # node_index: node index
    # node_dict: a dictionary that maps node index to coordinates
    # graph_type: 'grid', 'octagonal', or 'heavy-hex'

    # initialization
    node_coordinates = [0, 0]

    if graph_type == 'grid':
        node_pos = node_dict[node_index]
        node_coordinates = [node_pos[0], node_pos[1]]

    elif graph_type == 'octagonal':
        node_pos = node_dict[node_index][0:2]
        node_type = node_dict[node_index][2]
        # x-coordinate offset for different node types
        node_pos_x = [1/2, 1/2+1/np.sqrt(2), 1/2+1/np.sqrt(2), 1/2, -1/2, -1/2-1/np.sqrt(2), -1/2-1/np.sqrt(2), -1/2]
        # y-coordinate offset for different node types
        node_pos_y = [-1/2-1/np.sqrt(2), -1/2, 1/2, 1/2+1/np.sqrt(2), 1/2+1/np.sqrt(2), 1/2, -1/2, -1/2-1/np.sqrt(2)]
        node_coordinates = [node_pos[0] * (2 + np.sqrt(2)) + node_pos_x[node_type], node_pos[1] * (2 + np.sqrt(2)) + node_pos_y[node_type]]
    
    elif graph_type == 'heavy_hex':
        node_pos = node_dict[node_index][0:2]
        node_type = node_dict[node_index][2]
        # x-coordinate offset for different node types
        node_pos_x = [0, 1, 2, 3, 0]
        # y-coordinate offset for different node types
        node_pos_y = [0, 0, 0, 0, 1]
        node_coordinates = [(node_pos[0] + 1/2 * node_pos[1]) * 4 + node_pos_x[node_type], node_pos[1] * 2 + node_pos_y[node_type]]

    else:
        raise ValueError("Unsupported graph type.")

    return node_coordinates

def retrieve_multiple_node_coordinates(node_indices, node_dict, graph_type):
    # construct a mapping from node index to coordinates
    # node_indices: list of node indices
    # node_dict: a dictionary that maps node index to coordinates
    # graph_type: 'grid', 'octagonal', or 'heavy-hex'

    # retrieve position labels of nodes
    all_node_pos = np.array(list(node_dict.values()))
    array_node_pos = all_node_pos[np.array(node_indices)]

    if graph_type == 'grid':
        node_coordinates = array_node_pos[:, 0:2]

    elif graph_type == 'octagonal':
        node_pos = array_node_pos[:, 0:2]
        node_type = array_node_pos[:, 2]
        # x-coordinate offset for different node types
        node_pos_x = np.array([1/2, 1/2+1/np.sqrt(2), 1/2+1/np.sqrt(2), 1/2, -1/2, -1/2-1/np.sqrt(2), -1/2-1/np.sqrt(2), -1/2])
        # y-coordinate offset for different node types
        node_pos_y = np.array([-1/2-1/np.sqrt(2), -1/2, 1/2, 1/2+1/np.sqrt(2), 1/2+1/np.sqrt(2), 1/2, -1/2, -1/2-1/np.sqrt(2)])
        node_coordinates = node_pos * (2 + np.sqrt(2))
        node_coordinates[:, 0] += node_pos_x[node_type]
        node_coordinates[:, 1] += node_pos_y[node_type]
    
    elif graph_type == 'heavy_hex':
        node_pos = array_node_pos[:, 0:2]
        node_type = array_node_pos[:, 2]
        # x-coordinate offset for different node types
        node_pos_x = np.array([0, 1, 2, 3, 0])
        # y-coordinate offset for different node types
        node_pos_y = np.array([0, 0, 0, 0, 1])
        # node_coordinates = [(node_pos[0] + 1/2 * node_pos[1]) * 4 + node_pos_x[node_type], node_pos[1] * 2 + node_pos_y[node_type]]
        node_coordinates = np.zeros([len(node_type), 2]) # initialization
        node_coordinates[:, 0] += node_pos[:, 0] * 4 + node_pos[:, 1] * 2 + node_pos_x[node_type]
        node_coordinates[:, 1] += node_pos[:, 1] * 2 + node_pos_y[node_type]

    else:
        raise ValueError("Unsupported graph type.")

    return node_coordinates

In [156]:
def center_node_coordinates(graph_size, graph_type):
    # determine the position of the center of a graph with a specified size
    # graph_type: 'grid' or 'octagonal' or 'heavy_hex'
    # graph_size: [graph_width, graph_height]

    width = graph_size[0] # width of the graph
    height = graph_size[1] # height of the graph
    # initialization
    node_coordinates = [0, 0]

    if graph_type == 'grid':
        # position of the node in the center of the grid graph
        center_node_coordinates = [(width - 1) / 2, (height - 1) / 2]

    if graph_type == 'octagonal':
        # position of the node in the center of the octagonal graph
        center_node_coordinates = [(width - 1) / 2 * (2 + np.sqrt(2)), (height - 1) / 2 * (2 + np.sqrt(2))]

    if graph_type == 'heavy_hex':
        # position of the node in the center of the heavy-hex graph
        row_width = 4 * width + 3
        center_node_coordinates = [(row_width - 1) / 2, height]

    return center_node_coordinates

In [157]:
def normal_distribution_over_graph(node_index, node_dict, graph_type, graph_size, sigma, amplitude, offset):
    # calculate the value of a normal distribution, centered at the graph center, at the position of a qubit
    # node_index: node index
    # node_dict: a dictionary that maps node index to coordinates
    # graph_type: 'grid' or 'octagonal' or 'heavy_hex'
    # graph_size: [graph_width, graph_height]
    # sigma: standard deviation of the normal distribution
    # amplitude: coefficient of the normal distribution
    # offset: offset of the normal distribution

    pos_node = np.array(retrieve_node_coordinates(node_index, node_dict, graph_type))
    pos_center_node = np.array(center_node_coordinates(graph_size, graph_type))
    value = offset + amplitude * np.exp(-1/2 * (np.linalg.norm(pos_node-pos_center_node)**2) / (sigma**2))

    return value

def normal_distribution_over_graph_multiple_nodes(node_indices, node_dict, graph_type, graph_size, sigma, amplitude, offset):
    # calculate the value of a normal distribution, centered at the graph center, at the position of a qubit
    # node_indices: list of node indices
    # node_dict: a dictionary that maps node index to coordinates
    # graph_type: 'grid' or 'octagonal' or 'heavy_hex'
    # graph_size: [graph_width, graph_height]
    # sigma: standard deviation of the normal distribution
    # amplitude: coefficient of the normal distribution
    # offset: offset of the normal distribution

    pos_node = retrieve_multiple_node_coordinates(node_indices, node_dict, graph_type)
    pos_center_node = np.array(center_node_coordinates(graph_size, graph_type))
    value = offset + amplitude * np.exp(-1/2 * (np.linalg.norm(pos_node-pos_center_node, axis=(1))**2) / (sigma**2))

    return value

### Define quantum backend

In [159]:
import rustworkx as rx
# import numpy as np

class QuantumBackend:
    def __init__(self, num_qubits):
        self.num_qubits = num_qubits
        self.coupling_graph = None
        self.edge_list = None
        self.gate_types = {'rx': 1, 'ry': 1, 'cx': 2}
        self.gate_errors = {}
        self.readout_errors = np.zeros(self.num_qubits)

    def set_topology(self, coupling_graph):
        """Define qubit connectivity."""
        # if graph_type == 'grid':
        #     if size[0] * size[1] != self.num_qubits:
        #         raise ValueError("The provided grid size does not match the number of qubits.")
            
        #     self.coupling_graph = rx.generators.grid_graph(size[0], size[1]) # set a grid-type coupling graph
        #     self.edge_list = set(rx.generators.grid_graph(size[0], size[1]).edge_list())
        # else:
        #     raise ValueError("The specified graph type if not supported.")

        self.coupling_graph = coupling_graph
        self.num_qubits = coupling_graph.num_nodes()
        self.edge_list = set(coupling_graph.edge_list())

    def set_gate_types(self, gate_types):
        """Define the supported native gates."""
        self.gate_types = gate_types

    def set_gate_error(self, gate_name, qubit_indices, error_rate, reverse_flag=True):
        """Set error rate for a specific gate."""
        if gate_name not in self.gate_types:
            raise ValueError("The provided gate is not in the list of supported gate types.")

        if isinstance(qubit_indices, int):
            qubit_indices = [qubit_indices]

        # set error rate for single-qubit gate
        if len(qubit_indices) == 1:
            gate = gate_name + '_' + str(qubit_indices[0])
            self.gate_errors[gate] = error_rate

        # set error rate for two-qubit gate
        if len(qubit_indices) == 2:
            if tuple(qubit_indices) not in self.edge_list:
                raise ValueError("The provided qubits do not support a two-qubit gate.")
            gate = gate_name + '_' + str(qubit_indices[0]) + '_' + str(qubit_indices[1])
            self.gate_errors[gate] = error_rate
            if reverse_flag == True:
                gate = gate_name + '_' + str(qubit_indices[1]) + '_' + str(qubit_indices[0])
                self.gate_errors[gate] = error_rate

    def set_readout_error(self, qubit_index, error_rate):
        """Set error rate for a specific measurement."""
        self.readout_errors[qubit_index] = error_rate  # Replace 1.0 with the actual error probability

    def set_multiple_gate_error(self, gate_type, distribution='gaussian', seed=42, **kwargs):
        """Set error rates for all quantum gates of the same type, following a given distribution."""
        
        np.random.seed(seed)
        
        if distribution == 'gaussian':
            # the distribution of gate error rates satisfies a normal distribution
            gate_type = [gate_type, self.gate_types[gate_type]]
            if gate_type[1] == 1: # for single-qubit gate
                # error rates sampled from a gaussian distribution
                error_rates = abs(np.random.normal(kwargs['mean'], kwargs['std_dev'], self.num_qubits))
                for qubit_index in range(self.num_qubits):
                    self.set_gate_error(gate_name=gate_type[0], qubit_indices=qubit_index, error_rate=error_rates[qubit_index])
            if gate_type[1] == 2: # for two-qubit gate
                # error rates sampled from a gaussian distribution
                error_rates = abs(np.random.normal(kwargs['mean'], kwargs['std_dev'], len(self.edge_list)))
                edge_index = 0
                for edge in self.edge_list:
                    self.set_gate_error(gate_name=gate_type[0], qubit_indices=list(edge), error_rate=error_rates[edge_index])
                    edge_index = edge_index + 1

        if distribution == 'log_gaussian':
            # the log function of the gate error rate distribution satisfies a normal distribution
            gate_type = [gate_type, self.gate_types[gate_type]]
            if gate_type[1] == 1: # for single-qubit gate
                # log error rates sampled from a gaussian distribution
                log_error_rates = np.random.normal(kwargs['mean'], kwargs['std_dev'], self.num_qubits)
                error_rates = 10 ** log_error_rates
                for qubit_index in range(self.num_qubits):
                    self.set_gate_error(gate_name=gate_type[0], qubit_indices=qubit_index, error_rate=error_rates[qubit_index])
            if gate_type[1] == 2: # for two-qubit gate
                # log error rates sampled from a gaussian distribution
                log_error_rates = np.random.normal(kwargs['mean'], kwargs['std_dev'], len(self.edge_list))
                error_rates = 10 ** log_error_rates
                edge_index = 0
                for edge in self.edge_list:
                    self.set_gate_error(gate_name=gate_type[0], qubit_indices=list(edge), error_rate=error_rates[edge_index])
                    edge_index = edge_index + 1

        if distribution == 'normal':
            # the spatial distribution of gate error rates follows a normal distribution
            gate_type = [gate_type, self.gate_types[gate_type]]
            if gate_type[1] == 1: # for single-qubit gate
                for qubit_index in range(self.num_qubits):
                    error_rate = normal_distribution_over_graph(qubit_index, kwargs['node_dict'], kwargs['graph_type'], kwargs['graph_size'], kwargs['sigma'], kwargs['amplitude'], kwargs['offset'])
                    self.set_gate_error(gate_name=gate_type[0], qubit_indices=qubit_index, error_rate=error_rate)
            if gate_type[1] == 2: # for two-qubit gate
                for edge in self.edge_list:
                    error_rate_1 = normal_distribution_over_graph(edge[0], kwargs['node_dict'], kwargs['graph_type'], kwargs['graph_size'], kwargs['sigma'], kwargs['amplitude'], kwargs['offset'])
                    error_rate_2 = normal_distribution_over_graph(edge[1], kwargs['node_dict'], kwargs['graph_type'], kwargs['graph_size'], kwargs['sigma'], kwargs['amplitude'], kwargs['offset'])
                    self.set_gate_error(gate_name=gate_type[0], qubit_indices=[edge[0], edge[1]], error_rate=(error_rate_1+error_rate_2)/2)
        
    def set_multiple_readout_error(self, distribution='gaussian', seed=42, **kwargs):
        """Set error rates for all readout, following a given distribution."""
        
        np.random.seed(seed)

        if distribution == 'gaussian':
            # error rates sampled from a gaussian distribution
            error_rates = abs(np.random.normal(kwargs['mean'], kwargs['std_dev'], self.num_qubits))
            for qubit in range(self.num_qubits):
                self.set_readout_error(qubit_index=qubit, error_rate=error_rates[qubit])

        if distribution == 'log_gaussian':
            # log error rates sampled from a gaussian distribution
            log_error_rates = np.random.normal(kwargs['mean'], kwargs['std_dev'], self.num_qubits)
            error_rates = 10 ** log_error_rates
            for qubit in range(self.num_qubits):
                self.set_readout_error(qubit_index=qubit, error_rate=error_rates[qubit])

        if distribution == 'normal': 
            # the spatial distribution of readout error rates follows a normal distribution
            for qubit in range(self.num_qubits):
                error_rate = normal_distribution_over_graph(qubit, kwargs['node_dict'], kwargs['graph_type'], kwargs['graph_size'], kwargs['sigma'], kwargs['amplitude'], kwargs['offset'])
                self.set_readout_error(qubit_index=qubit, error_rate=error_rate)

    def gate_error(self, gate_name, qubit_indices):
        """Retrieve the gate error for a specific gate and qubit index."""
        if isinstance(qubit_indices, int):
            qubit_indices = [qubit_indices]

        if isinstance(qubit_indices, np.int64):
            qubit_indices = [int(qubit_indices)]

        # define the key for a gate acting on specific qubits
        if len(qubit_indices) == 1:
            gate = gate_name + '_' + str(qubit_indices[0])

        if len(qubit_indices) == 2:
            gate = gate_name + '_' + str(qubit_indices[0]) + '_' + str(qubit_indices[1])
        
        return self.gate_errors[gate]

    def readout_error(self, qubit_index):
        """Retrieve the readout error for a specific qubit index."""
        return self.readout_errors[qubit_index]

### Define quantum backend (support vectorization)

In [173]:
from scipy.sparse import csr_matrix

class QuantumBackendFast:
    """
    difference from the last quantum backend data structure: use 1D array to store the error rates of single-qubit gates, 
    sparse matrix to store the error rates of two-qubit gates, 
    batch lookup to efficiently retrieve the elements, 
    numpy array multiplication to improve the efficiency of calculating error rates.
    """
    def __init__(self, num_qubits):
        self.num_qubits = num_qubits
        self.coupling_graph = None
        self.edge_list = None
        self.gate_types = {'rx': 1, 'ry': 1, 'cx': 2}
        # save error rates of single-qubit gates as one-dimensional NumPy array, save those of two-qubit gates as SciPy sparse matrix
        self.gate_errors = {'rx': np.zeros(num_qubits), 'ry': np.zeros(num_qubits), 'cx': csr_matrix(([0], ([0], [0])), shape=(num_qubits, num_qubits))}
        self.readout_errors = np.zeros(num_qubits)

    def set_topology(self, coupling_graph):
        """Define qubit connectivity."""
        # if graph_type == 'grid':
        #     if size[0] * size[1] != self.num_qubits:
        #         raise ValueError("The provided grid size does not match the number of qubits.")
            
        #     self.coupling_graph = rx.generators.grid_graph(size[0], size[1]) # set a grid-type coupling graph
        #     self.edge_list = set(rx.generators.grid_graph(size[0], size[1]).edge_list())
        # else:
        #     raise ValueError("The specified graph type if not supported.")

        self.coupling_graph = coupling_graph
        self.num_qubits = coupling_graph.num_nodes()
        self.edge_list = set(coupling_graph.edge_list())

    def set_gate_types(self, gate_types):
        """Define the supported native gates."""
        self.gate_types = gate_types
        self.gate_errors = {}
        for gate_type in gate_types:
            if gate_types[gate_type] == 1:
                self.gate_errors[gate_type] = np.zeros(self.num_qubits)
            if gate_types[gate_type] == 2:
                self.gate_errors[gate_type] = csr_matrix(([0], ([0], [0])), shape=(self.num_qubits, self.num_qubits))

    def set_gate_error(self, gate_name, qubit_indices, error_rate, reverse_flag=True):
        """Set error rate for a specific gate."""
        if gate_name not in self.gate_types:
            raise ValueError("The provided gate is not in the list of supported gate types.")

        if isinstance(qubit_indices, int):
            qubit_indices = [qubit_indices]

        # set error rate for single-qubit gate
        if len(qubit_indices) == 1:
            gate_errors = np.array(self.gate_errors[gate_name])
            gate_errors[qubit_indices[0]] = error_rate
            self.gate_errors[gate_name] = gate_errors

        # set error rate for two-qubit gate
        if len(qubit_indices) == 2:
            if tuple(qubit_indices) not in self.edge_list:
                raise ValueError("The provided qubits are not connected, two-qubit gate cannot be performed.")
            
            gate_errors = csr_matrix(self.gate_errors[gate_name])
            gate_errors[qubit_indices[0], qubit_indices[1]] = error_rate
            if reverse_flag == True:
                gate_errors[qubit_indices[1], qubit_indices[0]] = error_rate
            self.gate_errors[gate_name] = gate_errors

    def set_readout_error(self, qubit_index, error_rate):
        """Set error rate for a specific measurement."""
        self.readout_errors[qubit_index] = error_rate  # Replace 1.0 with actual error rate of readout

    def set_multiple_gate_error(self, gate_type, distribution='gaussian', seed=42, reverse_flag=True, **kwargs):
        """Set error rates for all quantum gates of the same type, following a given distribution."""
        
        np.random.seed(seed)
        
        if distribution == 'gaussian':
            gate_size = self.gate_types[gate_type]
            if gate_size == 1: # for single-qubit gate
                # error rates sampled from a gaussian distribution
                error_rates = abs(np.random.normal(kwargs['mean'], kwargs['std_dev'], self.num_qubits))
                self.gate_errors[gate_type] = np.array(error_rates)
            if gate_size == 2: # for two-qubit gate
                # error rates sampled from a gaussian distribution
                if reverse_flag == False:
                    error_rates = abs(np.random.normal(kwargs['mean'], kwargs['std_dev'], len(self.edge_list)))
                    edge_list = np.transpose(np.array(list(self.edge_list)))
                    gate_errors = csr_matrix((error_rates, (edge_list[0], edge_list[1])), shape=(self.num_qubits, self.num_qubits))
                    self.gate_errors[gate_type] = gate_errors
                if reverse_flag == True:
                    error_rates = abs(np.random.normal(kwargs['mean'], kwargs['std_dev'], len(self.edge_list)))
                    edge_list = np.transpose(np.array(list(self.edge_list)))
                    gate_errors = csr_matrix((error_rates, (edge_list[0], edge_list[1])), shape=(self.num_qubits, self.num_qubits))
                    gate_errors[edge_list[1], edge_list[0]] = error_rates
                    self.gate_errors[gate_type] = gate_errors

        if distribution == 'log_gaussian':
            gate_size = self.gate_types[gate_type]
            if gate_size == 1: # for single-qubit gate
                # log error rates sampled from a gaussian distribution
                log_error_rates = np.random.normal(kwargs['mean'], kwargs['std_dev'], self.num_qubits)
                error_rates = 10 ** log_error_rates
                self.gate_errors[gate_type] = np.array(error_rates)
            if gate_size == 2: # for two-qubit gate
                # log error rates sampled from a gaussian distribution
                if reverse_flag == False:
                    log_error_rates = np.random.normal(kwargs['mean'], kwargs['std_dev'], len(self.edge_list))
                    error_rates = 10 ** log_error_rates
                    edge_list = np.transpose(np.array(list(self.edge_list)))
                    gate_errors = csr_matrix((error_rates, (edge_list[0], edge_list[1])), shape=(self.num_qubits, self.num_qubits))
                    self.gate_errors[gate_type] = gate_errors
                if reverse_flag == True:
                    log_error_rates = np.random.normal(kwargs['mean'], kwargs['std_dev'], len(self.edge_list))
                    error_rates = 10 ** log_error_rates
                    edge_list = np.transpose(np.array(list(self.edge_list)))
                    gate_errors = csr_matrix((error_rates, (edge_list[0], edge_list[1])), shape=(self.num_qubits, self.num_qubits))
                    gate_errors[edge_list[1], edge_list[0]] = error_rates
                    self.gate_errors[gate_type] = gate_errors
        
        if distribution == 'normal':
            # the spatial distribution of gate error rates follows a normal distribution
            gate_size = self.gate_types[gate_type]
            if gate_size == 1: # for single-qubit gate
                qubit_indices = list(range(self.num_qubits))
                error_rates = normal_distribution_over_graph_multiple_nodes(qubit_indices, kwargs['node_dict'], kwargs['graph_type'], kwargs['graph_size'], kwargs['sigma'], kwargs['amplitude'], kwargs['offset'])
                self.gate_errors[gate_type] = error_rates
            if gate_size == 2: # for two-qubit gate
                node_indices = np.array(list(backend.edge_list))
                error_rates_1 = normal_distribution_over_graph_multiple_nodes(node_indices[:, 0], kwargs['node_dict'], kwargs['graph_type'], kwargs['graph_size'], kwargs['sigma'], kwargs['amplitude'], kwargs['offset'])
                error_rates_2 = normal_distribution_over_graph_multiple_nodes(node_indices[:, 1], kwargs['node_dict'], kwargs['graph_type'], kwargs['graph_size'], kwargs['sigma'], kwargs['amplitude'], kwargs['offset'])
                gate_errors = csr_matrix(((error_rates_1+error_rates_2)/2, (node_indices[:, 0], node_indices[:, 1])), shape=(self.num_qubits, self.num_qubits))
                if reverse_flag == True:
                    gate_errors[node_indices[:, 1], node_indices[:, 0]] = (error_rates_1 + error_rates_2) / 2
                self.gate_errors[gate_type] = gate_errors
            
    def set_multiple_readout_error(self, distribution='gaussian', seed=42, **kwargs):
        """Set error rates for all readout, following a given distribution."""
        
        np.random.seed(seed)

        if distribution == 'gaussian':
            error_rates = abs(np.random.normal(kwargs['mean'], kwargs['std_dev'], self.num_qubits))
            self.readout_errors = error_rates
        if distribution == 'log_gaussian':
            log_error_rates = np.random.normal(kwargs['mean'], kwargs['std_dev'], self.num_qubits)
            error_rates = 10 ** log_error_rates
            self.readout_errors = error_rates
        if distribution == 'normal': # the spatial distribution of readout error rates follows a normal distribution
            qubit_indices = list(range(self.num_qubits))
            error_rates = normal_distribution_over_graph_multiple_nodes(qubit_indices, kwargs['node_dict'], kwargs['graph_type'], kwargs['graph_size'], kwargs['sigma'], kwargs['amplitude'], kwargs['offset'])
            self.readout_errors = error_rates

    def gate_error(self, gate_name, qubit_indices):
        """Retrieve the gate error for a specific gate and qubit index."""
        if isinstance(qubit_indices, int):
            qubit_indices = [qubit_indices]

        if isinstance(qubit_indices, np.int64):
            qubit_indices = [int(qubit_indices)]

        # define the key for a gate acting on specific qubits
        if len(qubit_indices) == 1:
            gate_errors = self.gate_errors[gate_name]
            gate_error = gate_errors[qubit_indices]

        if len(qubit_indices) == 2:
            gate_errors = self.gate_errors[gate_name]
            gate_error = gate_errors[qubit_indices[0], qubit_indices[1]]
        
        return gate_error

    def readout_error(self, qubit_index):
        """Retrieve the readout error for a specific qubit index."""
        return self.readout_errors[qubit_index]

### Define cost function/circuit mapping scoring function

In [162]:
def default_cost(circ, layouts, backend):
    """The default cost function of MAPOMATIC that returns the
    error rates across all the layouts for the given circuit

    Parameters:
        circ (QuantumCircuit): circuit of interest
        layouts (list of lists): List of specified layouts
        backend (QuantumQBackend): A quantum backend instance

    Returns:
        list: Tuples of layout and error
    """
    out = []
    # Make a single layout nested
    for layout in layouts:
        error = 0
        fid = 1
        for item in circ._data:
            if item[0].num_qubits == 2:
                q0 = circ.find_bit(item[1][0]).index # check this part
                q1 = circ.find_bit(item[1][1]).index
                fid *= (1-backend.gate_error(item[0].name, [layout[q0],
                                           layout[q1]]))

            elif item[0].name in ['h', 'x', 'u3']:
                q0 = circ.find_bit(item[1][0]).index
                fid *= 1-backend.gate_error(item[0].name, layout[q0])

            elif item[0].name in ['measure', 'reset']:
                q0 = circ.find_bit(item[1][0]).index
                fid *= 1-backend.readout_error(layout[q0])

        error = 1-fid
        out.append((layout, float(error)))
    return out

In [163]:
def default_cost_fast(circ, layouts, backend):
    """This function implements a fast calculation of the default cost function

    Parameters:
        circ (QuantumCircuit): circuit of interest
        layouts (list of lists): List of specified layouts
        backend (QuantumBackendFast): A quantum backend instance

    Returns:
        list: Tuples of layout and error
    """
    
    fid = np.ones(len(layouts))
    layouts = np.array(layouts)
    for item in circ._data:
        if item[0].num_qubits == 2: # for two-qubit gates
            q0 = circ.find_bit(item[1][0]).index # index of the first logical qubit
            q1 = circ.find_bit(item[1][1]).index # index of the second logical qubit
            physical_qubit_0 = layouts[:, q0] # index of the first physical qubit
            physical_qubit_1 = layouts[:, q1] # index of the second physical qubit
            gate_errors = backend.gate_errors[item[0].name] # the error rates of all two-qubit gates
            error_rates = gate_errors[physical_qubit_0, physical_qubit_1] # the error rates of the relevant two-qubit gates
            fid = fid * (1 - np.array(error_rates)[0])

        elif item[0].name in ['h', 'x', 'u3']: # for single-qubit gates
            q0 = circ.find_bit(item[1][0]).index # index of the logical qubit
            physical_qubit_0 = layouts[:, q0] # index of the physical qubit
            gate_errors = backend.gate_errors[item[0].name] # the error rates of all single-qubits gates
            error_rates = gate_errors[physical_qubit_0] # the error rates of the relevant single-qubit gates
            fid = fid * (1 - np.array(error_rates))

        elif item[0].name in ['measure', 'reset']:
            q0 = circ.find_bit(item[1][0]).index # same as the case of single-qubit gates
            physical_qubit_0 = layouts[:, q0]
            gate_errors = backend.readout_errors
            error_rates = gate_errors[physical_qubit_0]
            fid = fid * (1 - np.array(error_rates))
        error = 1-fid
    return error

In [164]:
def evaluate_layouts(circ, layouts, backend, cost_function=None):
    """Evaluate the error rate of the layout on a backend

    Parameters:
        circ (QuantumCircuit): circuit of interest
        layouts (list): Specified layouts
        backend (QuantumBackend): A quantum backend instance
        cost_function (callable): Custom cost function, default=None

    Returns:
        list: Tuples of layout, backend name, and cost
    """
    if not any(layouts):
        return []
    if not isinstance(layouts[0], list):
        layouts = [layouts]
    if cost_function is None:
        cost_function = default_cost
    out = cost_function(circ, layouts, backend)
    out.sort(key=lambda x: x[1])
    return out

In [165]:
def evaluate_layouts_fast(circ, layouts, backend, cost_function=None):
    """Efficiently evaluate the error rate of the layouts on a backend

    Parameters:
        circ (QuantumCircuit): circuit of interest
        layouts (list): Specified layouts
        backend (QuantumBackend): A quantum backend instance
        cost_function (callable): Custom cost function, default=None

    Returns:
        list: Tuples of layout, backend name, and cost
    """
    if not any(layouts):
        return []
    if not isinstance(layouts[0], list):
        layouts = [layouts]
    if cost_function is None:
        cost_function = default_cost_fast
    out = cost_function(circ, layouts, backend)
    sorted_layouts = sorted(zip(out, layouts))
    scores, layouts = zip(*sorted_layouts)
    return [scores, layouts]

In [166]:
def circuit_interaction_graph(circ):
    """Convert a circuit to the coupling graph it requires."""

    dag = circuit_to_dag(circ)
    qubits = dag.qubits
    qubit_indices = {qubit: index for index, qubit in enumerate(qubits)}

    interactions = []
    for node in dag.op_nodes(include_directives=False):
        len_args = len(node.qargs)
        if len_args == 2:
            interactions.append((qubit_indices[node.qargs[0]], qubit_indices[node.qargs[1]]))

    circ_interaction_graph = rx.PyGraph(multigraph=False)
    circ_interaction_graph.add_nodes_from(range(len(qubits)))
    circ_interaction_graph.add_edges_from_no_data(interactions)

    return circ_interaction_graph